In [1]:
import requests
import json
class ProdSysApi:
    
    def __init__(self, url_list, url_single, user, password):
        self._auth = requests.auth.HTTPBasicAuth(user, password)
        self._urls = [url_single, url_list]
        
    def api_url(self, id=None):
        if id is None:
            return self._urls[1]
        else:
            return self._urls[0].format(id)
    
    def list(self):
        response = requests.get(self.api_url(), auth=self._auth)
        print(len(response.content))
        data = response.json()
        return data
        
    def get(self, id):
        response = requests.get(self.api_url(id), auth=self._auth)
        data = response.json()
        return data
    
    def new(self, data):
            return requests.post(
                self.api_url(), 
                auth = self._auth,
                json = data,
            )
        
    def delete(self, id):
            return requests.delete(
                self.api_url(), 
                auth = self._auth,
            )
        
    def patch(self, id, data):
        if 'api/indesign/' in self._urls[0]:
            return requests.post(
                self.api_url(id), 
                auth = self._auth,
                data = {'json': json.dumps(data)},
            )
        else:
            return requests.patch(
                self.api_url(id), 
                auth = self._auth,
                json = data,
            )
            

        
old_api = ProdSysApi(
    user='api',
    password='Avier1gi',
    url_list = 'http://universitas.no/admin/api/indesign/json/',
    url_single = 'http://universitas.no/admin/api/indesign/{}/json/',
)
new_api = ProdSysApi(
    user='api',
    password='Avier1gi',
    url_list = 'http://universitas.no/api/legacy/?limit=500',
    url_single = 'http://universitas.no/api/legacy/{}/',
)
local_api = ProdSysApi(
    user='api',
    password='api',
    url_list = 'http://web:8000/api/legacy/?limit=10',
    url_single = 'http://web:8000/api/legacy/{}/',
)



In [2]:
prodsaker = [old_api.get(ps['prodsak_id']) for ps in old_api.list() if ps['produsert'] < '8']


488741


In [3]:
len(prodsaker)

89

In [13]:
from django.utils import timezone
def is_recent(sak):
    now = timezone.now()
    dato = timeparser(sak.get('version_date'))
    return now - dato < timezone.timedelta(days=30)
    
recent = sorted(filter(is_recent, prodsaker), reverse=True, key = lambda s:timeparser(s.get('version_date')))
for sak in recent:
    dato = timeparser(sak.get('version_date'))
    title = sak.get('arbeidstittel')
    print(dato, title)
    

2017-09-03 20:43:49+02:00 19 ANM The Book of Mormon
2017-09-03 20:15:36+02:00 19 KUL Arki
2017-09-03 18:07:13+02:00 19 urix Venezuela 2
2017-09-03 16:36:03+02:00 19 anm forsøksgym
2017-09-03 15:21:54+02:00 Melding hjem NYC 
2017-09-03 13:29:13+02:00 18+Anml+Lamegblimeddeg
2017-09-03 13:22:53+02:00 20 KUL Jenter og informatikk
2017-09-03 12:46:14+02:00 19 KUL ANB
2017-09-02 14:15:55+02:00 19 DEB forskerintervju
2017-09-01 22:11:42+02:00 19 DEB Svar til sparedama
2017-09-01 21:10:27+02:00 19 DEB nyhkom - studentvalget 2017
2017-09-01 13:30:15+02:00 Kvinnediskriminering i akademia
2017-08-30 08:29:40+02:00 18 MAG Politisk satire
2017-08-30 01:01:38+02:00 18 nyh pulenekt
2017-08-30 01:01:38+02:00 18nyhFengsletForsker utkast
2017-08-30 01:01:03+02:00 18 NYH Studenter bryr seg om utdanningspol
2017-08-30 00:55:48+02:00 17 DEB Trine Skei Grande
2017-08-30 00:55:25+02:00 18 Lille-leder
2017-08-30 00:47:53+02:00 18 KULKOM.kjønnssykdommer
2017-08-30 00:38:22+02:00 18 - Vi spør
2017-08-29 23:24:3

In [15]:
# Remove work in progress stories
#qs = Story.objects.filter(publication_status__lte=Story.STATUS_FROM_DESK)
#qs.delete()


def pre_save(model_instance, add):
    """ Returns field's value just before saving. """
    return getattr(model_instance, 'modified')

def update_timestamps(story, created, modified):
    story.created = created 
    story.modified = modified
    story._meta.get_field('modified').pre_save = pre_save
    story.save()


import pytz
def get_status(produsert, flagg, **kwargs):
    flag_status = {
        1: Story.STATUS_JOURNALIST,
        2: Story.STATUS_SUBEDITOR,
        3: Story.STATUS_EDITOR,
        4: Story.STATUS_EDITOR
    }.get(int(flagg))
    status = {
        Prodsak.DRAFT: flag_status,
        Prodsak.OLD_DRAFT: flag_status,
        Prodsak.READY_FOR_PRINT: Story.STATUS_TO_DESK,
        Prodsak.IMPORTED_TO_INDESIGN: Story.STATUS_AT_DESK,
        Prodsak.EXPORTED_FROM_INDESIGN: Story.STATUS_FROM_DESK,
    }.get(int(produsert))
    return status


def timeparser(value):
    dt=pytz.datetime.datetime.strptime(value, '%Y-%m-%d %H:%M:%S')
    return pytz.timezone('Europe/Oslo').localize(dt)

def import_all(saker, api):
    for ps in saker:
        ps = dict(**ps)
        ps['produsert'] = get_status(**ps)
        if ps['mappe'] == '\ufffdr':
            ps['mappe'] = 'år'
        response = api.new(ps)

        print('{prodsak_id} {mappe} {arbeidstittel} {response.status_code}'.format(**ps, response=response))
        if response.status_code == 201:
            data = response.json()
            created = timeparser(ps.get('dato'))
            modified = timeparser(ps.get('version_date'))
            diff = modified-created
            story = Story.objects.get(pk=data['prodsak_id'])
            update_timestamps(story, created, modified)
            story.refresh_from_db()
            diff2 = story.modified - story.created
            print(diff, diff==diff2)
        elif response.status_code == 400:
            ...
        else:
            print(response.status_code)
            print(json.dumps(ps, ensure_ascii=False, indent=2))
            error = response.text.split('Traceback (most recent call last):')[-1].strip()
            print(error.splitlines()[-1])
            print(error)
            qs = Story.objects.filter(publication_status__lt=Story.STATUS_FROM_DESK)
            qs.delete()
            break
            
import_all(recent[:], new_api)

23307 anmeldelser 19 ANM The Book of Mormon 400
23302 kultur 19 KUL Arki 400
23299 omverden 19 urix Venezuela 2 400
23308 anmeldelser 19 anm forsøksgym 201
0:00:00 True
23301 omverden Melding hjem NYC  201
0:00:00 True
23270 anmeldelser 18+Anml+Lamegblimeddeg 201
0:00:00 True
23306 kultur 20 KUL Jenter og informatikk 201
0:00:00 True
23303 kulturnotis 19 KUL ANB 201
0:00:00 True
23272 kronikk 19 DEB forskerintervju 201
0:00:00 True
23305 kronikk 19 DEB Svar til sparedama 201
0:00:00 True
23304 kommentar 19 DEB nyhkom - studentvalget 2017 201
0:00:00 True
23300 nyhet Kvinnediskriminering i akademia 201
0:00:00 True
23263 magasin 18 MAG Politisk satire 201
0:00:00 True
23287 nyhet 18 nyh pulenekt 201
7:29:18 True
23282 nyhet 18nyhFengsletForsker utkast 201
3:36:36 True
23285 nyhet 18 NYH Studenter bryr seg om utdanningspol 201
6:19:41 True
23232 kronikk 17 DEB Trine Skei Grande 201
8 days, 10:24:13 True
23294 nyhet 18 Lille-leder 201
4:53:26 True
23284 kulturkommentar 18 KULKOM.kjønnssyk

In [25]:
import json
import pprint
jsondump = r'''
{"count":16,"next":null,"previous":null,"results":[{"id":62586,"url":"http://universitas.no/api/stories/62586/","public_url":"http://universitas.no/nyheter/62586/","edit_url":"http://universitas.no/admin/stories/story/62586/change/","modified":"2017-09-03T17:52:48.007466Z","created":"2017-09-03T17:38:25.595855Z","working_title":"Nyhet 1","publication_status":100,"bodytext_markup":"1ER-SAKER ER SAKENE SOM PRIORITERES PÅ EN DOBBELTSIDE. DE ER ALLTID AVHENGIGE AV AT BILDET, ELLER ILLUSTRASJONEN ER BRA.\nSIDE 4-5: UTEN ANDRE SAKER: 4000 TEGN I HOVEDSAK, CA 1900 I UNDERSAK. MED ÉNSPALTER: 4000 – 4200. MED 2-SPALTER: 3800 – 4000. MED 3-SPALTER: CA 3800 TEGN. 8-SPALTER MED HEMS: CA 4300 TEGN. DOBBELTSIDE: MAKS 4200 TEGN I HOVEDSAK, CA 1900 I UNDERSAK. NB: MAKS 4200 TEGN I HOVEDSAKEN. BRUK UNDERSAK OM SAKEN SKAL VÆRE LENGRE ENN DETTE (MAKS 1900 TEGN).\n\n@stikktit:(PRØV Å UNNGÅ STIKKTITLER – STIKKTITLER ER SOM STØTTEHJUL!)\n\n@tit: \n\n@ing: \n\n@li:INGRESS1:\n\n@li:INGRESS2:\n@bt:\n\n@bt:\n\n@tema:\n\n@bl:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@sitat:\n@sitatbyline:\n\n@fakta:\n\n@tit:(undersak)\n\n@txt:(IKKE INGRESS I UNDERSAK)\n","story_type":2,"story_type_name":"Nyhet","byline_set":[]},{"id":62596,"url":"http://universitas.no/api/stories/62596/","public_url":"http://universitas.no/kultur/62596/","edit_url":"http://universitas.no/admin/stories/story/62596/change/","modified":"2017-09-03T17:51:59.431198Z","created":"2017-09-03T17:51:03.661346Z","working_title":"Kultur 3","publication_status":100,"bodytext_markup":"3ERSAKENE PÅ KULTUR ER SAKENE SOM TAR 50 % ELLER MINDRE AV PLASSEN PÅ EN SIDE, ELLER DOBBELTSIDE.\n2-SPALTER MED HEMS: CA 2800 TEGN. 3-SPALTER MED HEMS: CA 3500 TEGN (MÅ HA FOTOLØSNING). ÉNSPALTER UTEN HEMS: MAKS 2000 TEGN. HALVSIDE: 2600 – 2800 TEGN (MED STÅENDE BILDE). \n\n@tit:\n\n@ing:\n\n@bt:(SKRIV TI ORD TIL HVERT AV FRIMERKEBILDENE)\n\n@bt:\n\n@bl:\n\n@txt:Temaord:\n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@sitat:\n\n@sitatbyline:\n\n@txt:xx@universitas.no\n\n(LEGG ALLTID INN ETT ELLER TO FRIMERKEBILDER / TRYNEBILDER AV DE SOM UTTALER SEG I SAKEN).\n\n(NB! SELV OM DET KOMMER «WARNING» I PRODSYS NÅR @ STÅR I TEKSTEN, SKAL IKKE EPOSTADRESSE FJERNES. DET ER BARE «ERROR» SOM IKKE LAR DEG LAGRE)","story_type":10,"story_type_name":"Kultur","byline_set":[]},{"id":62595,"url":"http://universitas.no/api/stories/62595/","public_url":"http://universitas.no/kultur/62595/","edit_url":"http://universitas.no/admin/stories/story/62595/change/","modified":"2017-09-03T17:51:49.222941Z","created":"2017-09-03T17:50:57.659416Z","working_title":"Kultur 2","publication_status":100,"bodytext_markup":"2ERSAKENE PÅ KULTUR ER SAKENE SOM TAR MEST PLASS PÅ EN SIDE, ELLER TAT HELE SIDEN ALENE. SOM ALLTID: VELDIG AVHENGIG AV BRA FOTOLØSNING, KUL ILLUSTRASJON ELLER GRAFIKK.\nMED HEMS ELLER FEM PÅ PLASSEN: 2800 – 3000 TEGN. MED ÉNSPALTER: MAKS 3400. HELE SIDEN: MAKS 3800 TEGN.\n\n@tit:\n\n@ing:\n\n@bt:\n\n@bt:\n\n@bl:\n\n@txt:Temaord:\n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@sitat:\n\n@sitatbyline:\n\n@fakta:\n\n@txt:xx@universitas.no\n\n(NB! SELV OM DET KOMMER «WARNING» I PRODSYS NÅR @ STÅR I TEKSTEN, SKAL IKKE EPOSTADRESSE FJERNES. DET ER BARE «ERROR» SOM IKKE LAR DEG LAGRE)\n","story_type":10,"story_type_name":"Kultur","byline_set":[]},{"id":62585,"url":"http://universitas.no/api/stories/62585/","public_url":"http://universitas.no/kultur/62585/","edit_url":"http://universitas.no/admin/stories/story/62585/change/","modified":"2017-09-03T17:51:30.259825Z","created":"2017-09-03T17:38:21.380213Z","working_title":"Kultur 1","publication_status":100,"bodytext_markup":"1ERSAKENE TAR MER ENN 50 % PÅ EN DOBBELTSIDE. PÅ KULTUR KREVER DE MEGET GODE BILDER OG TEKSTLENGDE VARIERER VELDIG I FORHOLD TIL HVOR GODE BILDENE, GRAFIKKEN, ELLER ILLUSTRASJONEN ER.\nINNGANGSSIDE / DOBBELTSIDE MED HEMS: 7-SPALTER: 4500 – 4800 TEGN. 8-SPALTER: 4500 – 5000 TEGN. DOBBELTSIDE: 5000 – 6500 (VELDIG BILDEAVHENGIG).\n\n@tit:\n\n@ing:\n\n@bt:\n\n@bt:\n\n@bl:\n\n@txt:Temaord:\n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@sitat:\n\n@sitatbyline:\n\n@fakta:\n\n@txt:xx@universitas.no\n\n(NB! SELV OM DET KOMMER «WARNING» I PRODSYS NÅR @ STÅR I TEKSTEN, SKAL IKKE EPOSTADRESSE FJERNES. DET ER BARE «ERROR» SOM IKKE LAR DEG LAGRE)","story_type":10,"story_type_name":"Kultur","byline_set":[]},{"id":62593,"url":"http://universitas.no/api/stories/62593/","public_url":"http://universitas.no/nyheter/62593/","edit_url":"http://universitas.no/admin/stories/story/62593/change/","modified":"2017-09-03T17:50:46.527147Z","created":"2017-09-03T17:49:28.622573Z","working_title":"Nyhet 2","publication_status":100,"bodytext_markup":"2ER-SAKER ER SAKENE SOM TAR HELE ELLER MER ENN 50 % AV EN ENKELTSIDE. BILDE ELLER ILLUSTRASJON MÅ VÆRE TIPP TOPP.\nHELE SIDEN: 3500 – 4000 TEGN. MED ÉNSPALTER, FEM PÅ PLASSEN, ELLER HEMS: 3000 – 3500 TEGN.\n\n@stikktit:(PRØV Å UNNGÅ STIKKTITLER – STIKKTITLER ER SOM STØTTEHJUL!)\n\n@tit:\n\n@ing: \n @bt:\n\n@bt:\n\n@tema:\n\n@bl:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n @sitat:\n\n@sitatbyline:\n\n@fakta:\n @txt:xx@universitas.no\n\n (NB! SELV OM DET KOMMER «WARNING» I PRODSYS NÅR @ STÅR I TEKSTEN, SKAL IKKE EPOSTADRESSE FJERNES. DET ER BARE «ERROR» SOM IKKE LAR DEG LAGRE)\n\n","story_type":2,"story_type_name":"Nyhet","byline_set":[]},{"id":62594,"url":"http://universitas.no/api/stories/62594/","public_url":"http://universitas.no/nyheter/62594/","edit_url":"http://universitas.no/admin/stories/story/62594/change/","modified":"2017-09-03T17:50:42.830916Z","created":"2017-09-03T17:49:55.800802Z","working_title":"Nyhet 3","publication_status":100,"bodytext_markup":"3ERSAKER ER SAKENE SOM TAR MINDRE ENN HALVPARTEN AV SIDEN, ELLER DELER SIDE MED 50 % ELLER MER REKLAME.\n\nÉNSPALTER: 2000 TEGN (KUN SMÅ FRIMERKEBILDER). TOSPALTER: 3200 TEGN (LIGGENDE BILDE, ELLER FRIMERKEBILDE MED UTHEVET SITAT). HALVSIDER: 2600 – 2800 TEGN (BØR HA STÅENDE BILDE).\n\n@tit:KORT TITTEL \n\n@ing:\n\n@bt:(HUSK KJAPP BILDETEKST TIL FRIMERKEBILDENE OGSÅ. MAKS TI ORD)\n @temaord:\n\n@bl:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@mt:\n @sitat:\n\n@sitatbyline:\n @txt:xx@universitas.no\n\nHUSK Å FIKSE FRIMERKEBILDER / PRESSEBILDER AV DE SOM UTTALER SEG I SAKEN OG LEGG DE I NYHETSMAPPA PÅ DESKSERVERN. SKRIV HVEM DET ER I BILDETEKSTEN.\n\n(NB! SELV OM DET KOMMER «WARNING» I PRODSYS NÅR @ STÅR I TEKSTEN, SKAL IKKE EPOSTADRESSE FJERNES. DET ER BARE «ERROR» SOM IKKE LAR DEG LAGRE)\n","story_type":2,"story_type_name":"Nyhet","byline_set":[]},{"id":62592,"url":"http://universitas.no/api/stories/62592/","public_url":"http://universitas.no/side-2/62592/","edit_url":"http://universitas.no/admin/stories/story/62592/change/","modified":"2017-09-03T17:48:38.644466Z","created":"2017-09-03T17:48:23.595836Z","working_title":"Kommentar","publication_status":100,"bodytext_markup":"LENGDE: 3100 – 3200 TEGN\n\n@stikktit:\n\n@tit:\n\n@txt:\n\n@sitat:\n\n@bt:Illustrasjon:\n\n@bl:xx, journalist i Universitas \n\n@txt:xx@universitas.no\n","story_type":7,"story_type_name":"Kommentar","byline_set":[]},{"id":62591,"url":"http://universitas.no/api/stories/62591/","public_url":"http://universitas.no/feature/62591/","edit_url":"http://universitas.no/admin/stories/story/62591/change/","modified":"2017-09-03T17:48:17.007433Z","created":"2017-09-03T17:47:11.165539Z","working_title":"Min studietid","publication_status":100,"bodytext_markup":"FRA 2800 – 2900 TEGN\n\n@tit:\n\n@ing: [HUSK TABULATOR ]\nHVEM:\tnavn\nSTUDERTE:\tstudium\nNÅR:\t18XX-17XX\nAKTUELL MED:\tnoe greier\n\n@txt:\n\n@sitat:\n\n@bl:\n\n@txt:\n\n@bt:Foto: [kun dersom fotograf ikke er en av våre]\n\n","story_type":14,"story_type_name":"Min studietid","byline_set":[]},{"id":62584,"url":"http://universitas.no/api/stories/62584/","public_url":"http://universitas.no/anmeldelser/62584/","edit_url":"http://universitas.no/admin/stories/story/62584/change/","modified":"2017-09-03T17:47:33.106138Z","created":"2017-09-03T17:38:17.694753Z","working_title":"Anmeldelse","publication_status":100,"bodytext_markup":"@li:Verkstittel (TITTEL PÅ VERKET ELLER NAVNET PÅ ARTIST, HVIS DET ER KONSERT)\n\n@li:Av: xxx (BRUK TO ELLER TRE FAKTA OM DET SOM ANMELDES)\nForlag: xxx\nPlateselskap: xxx\nScene: xxx\nRegi: xxx\nMed: xxx\nTid: xxx\n\n@tit:Tittel på sak (MAKS 20 TEGN MED MELLOMROM) \n\n@tema:Sjanger (SKAL VÆRE GENERISK: PLATE, KONSERT, TIDSSKRIFT, UTSTILLING, FILM, ROMAN, SAKPROSA, NOVELLESAMLING, ETC.)\n\n@ing:Ingress (BARE PÅ STOR ANMELDELSE.70 TEGN PÅ EN LINJE, 140 PÅ TO)\n\n@txt:Brødtekst ( LITEN ANMELDELSE 1500 TEGN, STOR ANMELDELSE: 2200 TEGN)\n\n@bt:bildetekst (BARE PÅ STOR ANMELDELSE)\n\n@bl:Byline\n\n–\n\nOBS: TITTEL PÅ PLATER, BØKER OG FILMER SKAL SKRIVES I KURSIV\nTITTEL PÅ SANGER SKAL SKRIVES I «HERMETEGN» I BRØDTEKSTEN.\nI FAKTADELEN SKRIVES ALT RETT FRAM UTEN HERMETEGN\n","story_type":8,"story_type_name":"Anmeldelse","byline_set":[]},{"id":62590,"url":"http://universitas.no/api/stories/62590/","public_url":"http://universitas.no/annet/62590/","edit_url":"http://universitas.no/admin/stories/story/62590/change/","modified":"2017-09-03T17:47:08.130956Z","created":"2017-09-03T17:46:52.396407Z","working_title":"Quiz","publication_status":100,"bodytext_markup":"HUSK Å LEGGE BYLINEBILDE I DEBATTMAPPEN PÅ DESKEN\n\n@tit:\n\n@bl:\n\n@tema:\n\n@txt:\n","story_type":34,"story_type_name":"Quiz","byline_set":[]},{"id":62582,"url":"http://universitas.no/api/stories/62582/","public_url":"http://universitas.no/side-2/62582/","edit_url":"http://universitas.no/admin/stories/story/62582/change/","modified":"2017-09-03T17:45:48.904406Z","created":"2017-09-03T17:30:47.884184Z","working_title":"Kommentar","publication_status":100,"bodytext_markup":"LENGDE: 3100 – 3200 TEGN\n\n@stikktit:\n@tit:\n@txt:\n@sitat:\n@bt:Illustrasjon:\n@bl:xx, journalist i Universitas \n","story_type":7,"story_type_name":"Kommentar","byline_set":[]},{"id":62581,"url":"http://universitas.no/api/stories/62581/","public_url":"http://universitas.no/baksiden/62581/","edit_url":"http://universitas.no/admin/stories/story/62581/change/","modified":"2017-09-03T17:45:37.506553Z","created":"2017-09-03T17:29:50.073865Z","working_title":"Vi Spør","publication_status":100,"bodytext_markup":"LENGDE: 3100 – 3200 TEGN\n\n@stikktit:\n@tit:\n@spm: – spørsmål\n@txt: – svar\n\n@spm: – spørsmål\n@txt: – svar\n\n@spm: – spørsmål\n@txt: – svar\n\n@spm: – spørsmål\n@txt: – svar\n\n@sitat:\n@bt:Illustrasjon:\n@bl:xx","story_type":1,"story_type_name":"Vi spør","byline_set":[]},{"id":62589,"url":"http://universitas.no/api/stories/62589/","public_url":"http://universitas.no/side-2/62589/","edit_url":"http://universitas.no/admin/stories/story/62589/change/","modified":"2017-09-03T17:44:29.204893Z","created":"2017-09-03T17:44:15.400334Z","working_title":"Leder","publication_status":100,"bodytext_markup":"2200 TEGN MED ÉN LEDER. CA 2100 TIL SAMMEN MED TO ELLER MED SITAT.\n\n@tit:\n@txt:\n@sitat:\n\n@tit:\n@txt:\n\n","story_type":11,"story_type_name":"Leder","byline_set":[]},{"id":62588,"url":"http://universitas.no/api/stories/62588/","public_url":"http://universitas.no/anmeldelser/62588/","edit_url":"http://universitas.no/admin/stories/story/62588/change/","modified":"2017-09-03T17:42:30.328837Z","created":"2017-09-03T17:42:00.244579Z","working_title":"Ukas Advarsel / Anbefaling","publication_status":100,"bodytext_markup":"@tit: Tittel (OPPTIL 40 TEGN)\n\n@li: Hva er det? (ETT ORD)\n\n@li: Hvem: xxx\nHvor: xxx\nNår:  xxx (fak 1 og fak 2 SKAL VÆRE MAX 70 TEGN TIL SAMMEN)\n\n@txt: CIRKA 630 TEGN\n\n@bl:xx, journalist i Universitas\n\n–  \nHUSK Å TA BYLINEBILDE OG HUSK Å IKKE VÆRE FOR GENERELL.\n\nLAGRES SOM ARITKKELTYPE «TO UTVALGTE», HA MED ORDET «ADVARSEL» ELLER «ANBEFALING» I ARBEIDSTITTELEN.","story_type":39,"story_type_name":"Ukas Advarsel","byline_set":[]},{"id":62583,"url":"http://universitas.no/api/stories/62583/","public_url":"http://universitas.no/baksiden/62583/","edit_url":"http://universitas.no/admin/stories/story/62583/change/","modified":"2017-09-03T17:41:47.429176Z","created":"2017-09-03T17:38:11.695631Z","working_title":"Ad Notam","publication_status":100,"bodytext_markup":"@tit: \n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@bt:\n\n@bt:\n\n–\nETT ELLER TO BILDER I HØYDEFORMAT.\nMED ETT BILDE: 2500 TEGN (inkl. Mellomrom)\nMED TO BILDER: 2400 TEGN (inkl. Mellomrom)","story_type":15,"story_type_name":"Ad Notam","byline_set":[]},{"id":62587,"url":"http://universitas.no/api/stories/62587/","public_url":"http://universitas.no/nyheter/62587/","edit_url":"http://universitas.no/admin/stories/story/62587/change/","modified":"2017-09-03T17:40:35.900638Z","created":"2017-09-03T17:38:30.167695Z","working_title":"Fem på plassen","publication_status":100,"bodytext_markup":"@ing:Spørsmål (IKKE LEDENDE!!)\n\n@mt:Navn (alder)\n@txt:skole, fag\n@txt:MAX 150 TEGN\n\n@mt:Navn (alder)\n@txt:skole, fag\n@txt:MAX 150 TEGN\n\n@mt:Navn (alder)\n@txt:skole, fag\n@txt:MAX 150 TEGN\n\n@mt:Navn (alder)\n@txt:skole, fag\n@txt:MAX 150 TEGN\n\n@mt:Navn (alder)\n@txt:skole, fag\n@txt:MAX 150 TEGN\n\n@mt:Navn (alder)\n@txt:skole, fag\n@txt:MAX 150 TEGN\n\n","story_type":40,"story_type_name":"Fem på plassen","byline_set":[]}]}
'''
data = json.loads(jsondump)
for story in data["results"]:
    postdata = {k:story[k] for k in ('publication_status', 'story_type', 'working_title','bodytext_markup')}
    postdata = dict(
        produsert=story['publication_status'], 
        arbeidstittel=story['working_title'],
        tekst=story['bodytext_markup'],
        mappe='news',
    )
    resp = new_api.new(postdata)
    print(resp.status_code, resp.json())

201 {'bilete': [], 'prodsak_id': 62613, 'mappe': 'news', 'arbeidstittel': 'Nyhet 1', 'produsert': 100, 'tekst': '1ER-SAKER ER SAKENE SOM PRIORITERES PÅ EN DOBBELTSIDE. DE ER ALLTID AVHENGIGE AV AT BILDET, ELLER ILLUSTRASJONEN ER BRA.\nSIDE 4-5: UTEN ANDRE SAKER: 4000 TEGN I HOVEDSAK, CA 1900 I UNDERSAK. MED ÉNSPALTER: 4000 – 4200. MED 2-SPALTER: 3800 – 4000. MED 3-SPALTER: CA 3800 TEGN. 8-SPALTER MED HEMS: CA 4300 TEGN. DOBBELTSIDE: MAKS 4200 TEGN I HOVEDSAK, CA 1900 I UNDERSAK. NB: MAKS 4200 TEGN I HOVEDSAKEN. BRUK UNDERSAK OM SAKEN SKAL VÆRE LENGRE ENN DETTE (MAKS 1900 TEGN).\n\n@stikktit:(PRØV Å UNNGÅ STIKKTITLER – STIKKTITLER ER SOM STØTTEHJUL!)\n\n@tit: \n\n@ing: \n\n@li:INGRESS1:\n\n@li:INGRESS2:\n@bt:\n\n@bt:\n\n@tema:\n\n@bl:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@sitat:\n@sitatbyline:\n\n@fakta:\n\n@tit:(undersak)\n\n@txt:(IKKE INGRESS I UNDERSAK)', 'url': 'http://universitas.no/api/legacy/62613/', 'version_no': 1}
201 {'bilete': [], 'prodsak_id': 62614, 'mappe': 'news

201 {'bilete': [], 'prodsak_id': 62627, 'mappe': 'news', 'arbeidstittel': 'Ad Notam', 'produsert': 100, 'tekst': '@tit: \n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@bt:\n\n@bt:\n\n–\nETT ELLER TO BILDER I HØYDEFORMAT.\nMED ETT BILDE: 2500 TEGN (inkl. Mellomrom)\nMED TO BILDER: 2400 TEGN (inkl. Mellomrom)', 'url': 'http://universitas.no/api/legacy/62627/', 'version_no': 1}
201 {'bilete': [], 'prodsak_id': 62628, 'mappe': 'news', 'arbeidstittel': 'Fem på plassen', 'produsert': 100, 'tekst': '@ing:Spørsmål (IKKE LEDENDE!!)\n\n@mt:Navn (alder)\n@txt:skole, fag\n@txt:MAX 150 TEGN\n\n@mt:Navn (alder)\n@txt:skole, fag\n@txt:MAX 150 TEGN\n\n@mt:Navn (alder)\n@txt:skole, fag\n@txt:MAX 150 TEGN\n\n@mt:Navn (alder)\n@txt:skole, fag\n@txt:MAX 150 TEGN\n\n@mt:Navn (alder)\n@txt:skole, fag\n@txt:MAX 150 TEGN\n\n@mt:Navn (alder)\n@txt:skole, fag\n@txt:MAX 150 TEGN', 'url': 'http://universitas.no/api/legacy/62628/', 'version_no': 1}


In [27]:

Story._meta.get_field('modified').pre_save = pre_save

for story in Story.objects.filter(publication_status__lt=Story.STATUS_PUBLISHED):
    data = old_api.get(story.pk)
    created = parser(data.get('dato'))
    modified = parser(data.get('version_date'))
    story.created = created
    story.modified = modified
    story.publication_status = flags[data.get('flagg')]
    story.save(update_fields=['created', 'modified'] )
    print(story)
    print(created, modified, story.modified, sep='\n')

NameError: name 'parser' is not defined

In [ ]:
import pytz
pytz.country_timezones('no')

In [ ]:
def get_story_type(prodsys_mappe):
    """ Find correct story type for the story based on legacy label. """
    try:
        story_type = StoryType.objects.filter(prodsys_mappe=prodsys_mappe)[0]
    except IndexError:
        generic_section, exists = Section.objects.get_or_create(
            title='New Section')
        story_type = StoryType.objects.create(
            prodsys_mappe=prodsys_mappe,
            name="New Story Type ({label})".format(label=prodsys_mappe),
            section=generic_section,
        )
    # finally:
    return story_type

In [ ]:
Story.objects.filter(publication_status__lte=1).delete()

In [ ]:
response = api.post(id=23215, data={'arbeidstittel': 'awesome', 'produsert': 0})
print(response.request.headers, response.request.body, sep='\n')
print(response.url, response.status_code, response.text)
testsak = api.get(23215)
print(testsak['arbeidstittel'], testsak['version_no'], testsak['produsert'])

In [ ]:
from django.contrib.auth.models import Permission
pp = Permission.objects.filter(codename__contains='_story')
for p in pp: print(p.codename)jjj

In [ ]:
from django.contrib.auth import authenticate
create_user = get_user_model().objects.create
get_user_model().objects.filter(username='api').delete()
api_user = get_user_model().objects.create_user(
    username='api',
    password='api',
    email='api@api.com',
)
print(api_user)
login = authenticate(username='api', password='api')
print(login)

In [ ]:
from django.contrib.auth import get_user_model

def api_user():
    User = get_user_model()
    try:
        user = User.objects.get(username='api')
    except User.DoesNotExist:
        user = User.objects.create_user(
            username='api',
            password='api',
            email='api@example.com',
        )
    permissions = list(Permission.objects.filter(codename__contains='_story'))
    user.user_permissions.add(*permissions)
    for p in permissions:
        print(p.codename)
    return user

api_user()

In [ ]:
saker = api.list()
til_web = list(filter(lambda sak: sak['produsert'] == '6', saker))
pprint(til_web[7])


In [ ]:
pprint(api.get(23194))

In [ ]:
def deserialize_story(story):
    output = [] 
    head_tags = [('tit', 'title'),('ing', 'lede'),('tema', 'theme_word'),('stikktit', 'kicker')]
    for tag, attr in head_tags:
        text = getattr(story, attr)
        if text:
            output.append(f'@{tag}:{text}')
    for bl in story.byline_set.select_related('contributor'):
        output.append(str(bl))
    output.append(story.bodytext_markup) 
    for aside in story.asides():
        output.append(aside.child.bodytext_markup)
    for pullquote in story.pullquotes():
        output.append(pullquote.child.bodytext_markup)
    return '\n'.join(output)
    
story = Story.objects.order_by('?').last()
txt1 = deserialize_story(story)
prodstory = api.get(story.prodsak_id)
txt2 = prodstory.get('tekst')
print(story.prodsak_id)
print(txt1)
print(txt2)

In [ ]:
story = Story.objects.get(id=61090)
story.images()

In [ ]:
imgstories = Story.objects.filter(pk__in=StoryImage.objects.all().values_list('parent_story', flat=True).distinct())
imgstories[0].pk
